## 1. 크롤링과 데이터베이스 예제

* cine21 배우 랭킹 사이트 크롤링
    - 사이트 주소 : http://www.cine21.com/rank/person/
    - 요청 방식 확인 방법 : 크롬 개발자 모드로 들어가서, Network -> content 페이지의 요청 방식 확인
        - Requests Url: http://www.cine21.com/rank/person/content
        - Requests Method : POST
        - Form data
            - section = 'actor'
            - period_start: 2020-11
            - gender: all
            - page: 1

1.  라이브러리 import

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

### 2. mongodb connection

In [2]:
conn = pymongo.MongoClient()
actor_db = conn['cine21']
actor_collection = actor_db['actor_collection']


#### 3. 크롤링 주소 requests

In [3]:
url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2020-11'
post_data['gender'] = 'all'
post_data['page'] = 1

res = requests.post(url, data=post_data)

### 4. parsing  과 데이터 추출

In [4]:
soup = BeautifulSoup(res.content, 'html.parser')

In [5]:
actors = soup.select("li.people_li  div.name")
for actor in actors:
#     print(actor.get_text().split("(")[0])
    print(re.sub('\([A-Za-z0-9가-힣]*\)','',actor.get_text()))

이제훈
조우진
신혜선
한예리
김영광
윤여정
임원희


In [26]:
test_data = '주지훈(6편)'
re.sub('\([A-Za-z0-9가-힣]*\)','',test_data)

'주지훈'

### 5. 배우 상세정보 추출


#### 참고: 특수한 정규 표현식
    -Greedy(.*) vs Non-Greedy(.*?)
    - https://regexr.com
        -<li><span class="tit">생년월일</span>1984-07-04</li>
    - . 문자는 줄바꿈 문자인 \n 를 제외한 모든 문자 한개를 의미함
    - * 는 앞 문자가 0번 또는 그 이상 반복되는 패턴
    - .*

In [18]:
actors = soup.select("li.people_li  div.name")

actors_info_list = list()
for actor in actors:
    actor_link = 'http://www.cine21.com' + actor.select_one("a").attrs['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
    default_info = soup_actor.select_one("ul.default_info")
    actor_details = default_info.select('li')
    
    actor_info_dict = dict()
    
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').get_text()
        actor_item_value = re.sub('<span.*?>.*</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value).replace('\n', '')
        actor_info_dict[actor_item_field] = actor_item_value
        
    actors_info_list.append(actor_info_dict)
actors_info_list

[{'직업': '배우',
  '생년월일': '1984-07-04',
  '성별': '남',
  '홈페이지': 'https://www.facebook.com/saram.leejehoon',
  '신장/체중': '176cm',
  '학교': '한국예술종합학교 연극원 연기과',
  '특기': '댄스, 노래, 무술',
  '소속사': '㈜사람엔터테인먼트'},
 {'다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'},
 {'직업': '배우',
  '생년월일': '1989-00-00',
  '성별': '여',
  '홈페이지': 'https://www.facebook.com/shs831 https://www.instagram.com/shinhs831/'},
 {'직업': '배우',
  '생년월일': '1984-12-23',
  '성별': '여',
  '홈페이지': 'https://www.facebook.com/saram.hanyeri',
  '학교': '한국예술종합학교'},
 {'직업': '배우',
  '생년월일': '1987-01-11',
  '성별': '남',
  '홈페이지': 'https://twitter.com/aksakfn12https://www.instagram.com/aksakfn12/'},
 {'원어명': '윤여정',
  '직업': '배우',
  '생년월일': '1947-06-19',
  '성별': '여',
  '학교': '한양대학교 국문학 (중퇴)'},
 {'직업': '배우',
  '생년월일': '1970-10-11',
  '성별': '남',
  '신장/체중': '175cm, 68kg',
  '학교': '서울예술대학 연극',
  '소속사': '빌리지엔터테인먼트'}]

#### 6. 흥행 지수 추출

In [56]:
actors = soup.select("li.people_li  div.name")
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')
movies_list = list()
for index, actor in enumerate(actors):
    movies_dict = dict()
    movie_actor = re.sub('\(.*\)', '', actor.get_text())
    movie_value = int(hits[index].get_text().replace(',',''))
    movie_titles = movies[index].select('li a span')
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.get_text())
    print(movie_title_list)
    print()

이제훈
흥행지수: 100812
['도굴', '건축학개론', '파수꾼', '박열']

조우진
흥행지수: 88076
['도굴', '서복', '국가부도의 날', '강철비']

신혜선
흥행지수: 50390
['도굴']

한예리
흥행지수: 47424
['미나리', '최악의 하루', '더 테이블', '춘몽']

김영광
흥행지수: 44812
['미션 파서블']

윤여정
흥행지수: 39673
['화녀', '미나리', '지푸라기라도 잡고 싶은 짐승들', '찬실이는 복도 많지', '죽여주는 여자', '그것만이 내 세상']

임원희
흥행지수: 25245
['도굴', '신과 함께-죄와 벌', '신과 함께-인과 연', '천문: 하늘에 묻는다']



#### 7. 수집된 데이터 기반 사전만들기

In [58]:

actors = soup.select("li.people_li  div.name")
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')

actors_info_list = list()

for index, actor in enumerate(actors):
    actor_name = re.sub('\(.*\)', '', actor.get_text())
    actor_hits = int(hits[index].get_text().replace(',',''))
    movie_titles = movies[index].select('li a span')
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.get_text())
    actor_info_dict = dict()
    actor_info_dict['배우이름'] = actor_name
    actor_info_dict['흥행지수'] = actor_hits
    actor_info_dict['출연영화'] = movie_title_list
    
    actor_link = 'http://www.cine21.com' + actor.select_one("a").attrs['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
    default_info = soup_actor.select_one("ul.default_info")
    actor_details = default_info.select('li')
    
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').get_text()
        actor_item_value = re.sub('<span.*?>.*</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value).replace('\n', '')
        actor_info_dict[actor_item_field] = actor_item_value
        
    actors_info_list.append(actor_info_dict)
actors_info_list

[{'배우이름': '이제훈',
  '흥행지수': 100812,
  '출연영화': ['도굴', '건축학개론', '파수꾼', '박열'],
  '직업': '배우',
  '생년월일': '1984-07-04',
  '성별': '남',
  '홈페이지': 'https://www.facebook.com/saram.leejehoon',
  '신장/체중': '176cm',
  '학교': '한국예술종합학교 연극원 연기과',
  '특기': '댄스, 노래, 무술',
  '소속사': '㈜사람엔터테인먼트'},
 {'배우이름': '조우진',
  '흥행지수': 88076,
  '출연영화': ['도굴', '서복', '국가부도의 날', '강철비'],
  '다른 이름': '조신제',
  '직업': '배우',
  '생년월일': '1979-01-16',
  '성별': '남'},
 {'배우이름': '신혜선',
  '흥행지수': 50390,
  '출연영화': ['도굴'],
  '직업': '배우',
  '생년월일': '1989-00-00',
  '성별': '여',
  '홈페이지': 'https://www.facebook.com/shs831 https://www.instagram.com/shinhs831/'},
 {'배우이름': '한예리',
  '흥행지수': 47424,
  '출연영화': ['미나리', '최악의 하루', '더 테이블', '춘몽'],
  '직업': '배우',
  '생년월일': '1984-12-23',
  '성별': '여',
  '홈페이지': 'https://www.facebook.com/saram.hanyeri',
  '학교': '한국예술종합학교'},
 {'배우이름': '김영광',
  '흥행지수': 44812,
  '출연영화': ['미션 파서블'],
  '직업': '배우',
  '생년월일': '1987-01-11',
  '성별': '남',
  '홈페이지': 'https://twitter.com/aksakfn12https://www.instagram.com/aksakfn12/'},
 {'배우이

#### 8. 여러 페이지의  배우 상세정보 추출하기

In [67]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re
actors_info_list = list()

url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2020-11'
post_data['gender'] = 'all'
for index in range(1, 21):
    post_data['page'] = index

    res = requests.post(url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')


    actors = soup.select("li.people_li")
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')

    for index, actor in enumerate(actors):
        actor_name = re.sub('\(.*\)', '', actor.select_one('div.name').get_text())
        actor_hits = int(hits[index].get_text().replace(',',''))
        movie_titles = movies[index].select('li a span')
        ranking = actor.select_one('span.grade').get_text()
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.get_text())
        actor_info_dict = dict()
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_title_list
        actor_info_dict['랭킹'] = ranking

        actor_link = 'http://www.cine21.com' + actor.select_one("a").attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one("ul.default_info")
        actor_details = default_info.select('li')

        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').get_text()
            actor_item_value = re.sub('<span.*?>.*</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value).replace('\n', '')
            actor_info_dict[actor_item_field] = actor_item_value

        actors_info_list.append(actor_info_dict)
actors_info_list

#### 9. mongodb에 크롤링 데이터 저장히기

In [63]:
conn = pymongo.MongoClient()
actor_db = conn['cine21']
actor_collection = actor_db['actor_collection']
actor_collection.insert_many(actors_info_list)

In [106]:
actor_collection.delete_many({})

#### 10. 배우 랭킹 및 상세 정보 크롤링 및 mongdo 저장

In [107]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn['cine21']
actor_collection = actor_db['actor_collection']

actors_info_list = list()

url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2020-11'
post_data['gender'] = 'all'
for index in range(1, 21):
    post_data['page'] = index

    res = requests.post(url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')


    actors = soup.select("li.people_li")
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')

    for index, actor in enumerate(actors):
        actor_name = re.sub('\(.*\)', '', actor.select_one('div.name').get_text())
        actor_hits = int(hits[index].get_text().replace(',',''))
        movie_titles = movies[index].select('li a span')
        ranking = actor.select_one('span.grade').get_text()
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.get_text())
        actor_info_dict = dict()
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_title_list
        actor_info_dict['랭킹'] = ranking

        actor_link = 'http://www.cine21.com' + actor.select_one("a").attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one("ul.default_info")
        actor_details = default_info.select('li')

        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').get_text()
            actor_item_value = re.sub('<span.*?>.*</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value).replace('\n', '')
            actor_info_dict[actor_item_field] = actor_item_value

        actors_info_list.append(actor_info_dict)
actor_collection.insert_many(actors_info_list)

In [69]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn['cine21']
actor_collection = actor_db['actor_collection']

actor_collection.find_one({})
docs = actor_collection.find({}).limit(3)
for doc in docs:
    print(doc)

{'_id': ObjectId('608fcb7abc196d809d52e100'), '배우이름': '이제훈', '흥행지수': 100812, '출연영화': ['도굴', '건축학개론', '파수꾼', '박열'], '랭킹': '1', '직업': '배우', '생년월일': '1984-07-04', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.leejehoon', '신장/체중': '176cm', '학교': '한국예술종합학교 연극원 연기과', '특기': '댄스, 노래, 무술', '소속사': '㈜사람엔터테인먼트'}
{'_id': ObjectId('608fcb7abc196d809d52e101'), '배우이름': '조우진', '흥행지수': 88076, '출연영화': ['도굴', '서복', '국가부도의 날', '강철비'], '랭킹': '2', '다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'}
{'_id': ObjectId('608fcb7abc196d809d52e102'), '배우이름': '신혜선', '흥행지수': 50390, '출연영화': ['도굴'], '랭킹': '3', '직업': '배우', '생년월일': '1989-00-00', '성별': '여', '홈페이지': 'https://www.facebook.com/shs831 https://www.instagram.com/shinhs831/'}


In [70]:
actor_collection.update_many({},  { '$rename' : {'흥행지수':'흥행 지수' } } )

In [108]:
docs = actor_collection.find({}).limit(3)
for doc in docs:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e21c'), '배우이름': '이제훈', '흥행지수': 100812, '출연영화': ['도굴', '건축학개론', '파수꾼', '박열'], '랭킹': '1', '직업': '배우', '생년월일': '1984-07-04', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.leejehoon', '신장/체중': '176cm', '학교': '한국예술종합학교 연극원 연기과', '특기': '댄스, 노래, 무술', '소속사': '㈜사람엔터테인먼트'}
{'_id': ObjectId('608ffdadbc196d809d52e21d'), '배우이름': '조우진', '흥행지수': 88076, '출연영화': ['도굴', '서복', '국가부도의 날', '강철비'], '랭킹': '2', '다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'}
{'_id': ObjectId('608ffdadbc196d809d52e21e'), '배우이름': '신혜선', '흥행지수': 50390, '출연영화': ['도굴'], '랭킹': '3', '직업': '배우', '생년월일': '1989-00-00', '성별': '여', '홈페이지': 'https://www.facebook.com/shs831 https://www.instagram.com/shinhs831/'}


## 참고 : collection 변수명 변경

In [110]:
actor = actor_collection

In [74]:
docs = actor_collection.find({}).sort('생년월일', pymongo.DESCENDING).limit(10) # 반대로 솔팅하고 싶을때 
for doc in docs:
    print(doc)

{'_id': ObjectId('608fcb7abc196d809d52e153'), '배우이름': '박소이', '출연영화': ['담보'], '랭킹': '84', '직업': '배우', '생년월일': '2012-03-12', '성별': '여', '흥행 지수': 1258}
{'_id': ObjectId('608fcbf5bc196d809d52e1e0'), '배우이름': '박소이', '출연영화': ['담보'], '랭킹': '84', '직업': '배우', '생년월일': '2012-03-12', '성별': '여', '흥행 지수': 1258}
{'_id': ObjectId('608fcb7abc196d809d52e136'), '배우이름': '정현준', '출연영화': ['이웃사촌', '기생충'], '랭킹': '55', '직업': '배우', '생년월일': '2011-11-08', '성별': '남', '흥행 지수': 3374}
{'_id': ObjectId('608fcbf5bc196d809d52e1c3'), '배우이름': '정현준', '출연영화': ['이웃사촌', '기생충'], '랭킹': '55', '직업': '배우', '생년월일': '2011-11-08', '성별': '남', '흥행 지수': 3374}
{'_id': ObjectId('608fcb7abc196d809d52e134'), '배우이름': '김강훈', '출연영화': ['내일의 기억', '엑시트'], '랭킹': '53', '직업': '배우', '생년월일': '2009-06-07', '성별': '남', '홈페이지': 'http://instagram.com/kanghoon.juwon', '흥행 지수': 3667}
{'_id': ObjectId('608fcbf5bc196d809d52e1c1'), '배우이름': '김강훈', '출연영화': ['내일의 기억', '엑시트'], '랭킹': '53', '직업': '배우', '생년월일': '2009-06-07', '성별': '남', '홈페이지': 'http://instagram.com/kang

In [82]:
docs = actor_collection.find({'특기': {'$exists':True } } ).sort('흥행 지수').limit(5)
for doc in docs:
    print(doc)

{'_id': ObjectId('608fcbf5bc196d809d52e212'), '배우이름': '최철호', '출연영화': ['요가학원: 죽음의 쿤달리니'], '랭킹': '134', '직업': '배우', '생년월일': '1970-03-02', '성별': '남', '신장/체중': '182cm, 70kg', '학교': '대일외국어고등학교', '취미': '낚시, 여행, 자전거하이킹', '특기': '검도', '소속사': '오피스제로', '흥행 지수': 445}
{'_id': ObjectId('608fcb7abc196d809d52e185'), '배우이름': '최철호', '출연영화': ['요가학원: 죽음의 쿤달리니'], '랭킹': '134', '직업': '배우', '생년월일': '1970-03-02', '성별': '남', '신장/체중': '182cm, 70kg', '학교': '대일외국어고등학교', '취미': '낚시, 여행, 자전거하이킹', '특기': '검도', '소속사': '오피스제로', '흥행 지수': 445}
{'_id': ObjectId('608fcbf5bc196d809d52e1ed'), '배우이름': '정수정', '출연영화': ['애비규환'], '랭킹': '97', '다른 이름': 'f(x);fx;에프엑스 ; 크리스탈', '직업': '가수', '생년월일': '1994-10-24', '성별': '여', '홈페이지': 'https://instagram.com/vousmevoyez', '신장/체중': '163cm, 45kg', '학교': '한국켄트외국인학교 재학', '취미': '연기', '특기': '댄스', '소속사': 'SM엔터테인먼트', '흥행 지수': 936}
{'_id': ObjectId('608fcb7abc196d809d52e160'), '배우이름': '정수정', '출연영화': ['애비규환'], '랭킹': '97', '다른 이름': 'f(x);fx;에프엑스 ; 크리스탈', '직업': '가수', '생년월일': '1994-10-24', '성별': '여', '홈페이

In [89]:
docs = actor_collection.find({'생년월일': {'$exists' : False } }, {'배우이름':1, '_id':0} )
for doc in docs:
    print(doc)

{'배우이름': '고아성'}
{'배우이름': '이솜'}
{'배우이름': '남민우'}
{'배우이름': '하성용'}
{'배우이름': '하윤경'}
{'배우이름': '정재헌'}
{'배우이름': '신햇빛'}
{'배우이름': '이용규'}
{'배우이름': '김성민'}
{'배우이름': '김율'}
{'배우이름': '감소현'}
{'배우이름': '오인실'}
{'배우이름': '김정은'}
{'배우이름': '문승아'}
{'배우이름': '한경화'}
{'배우이름': '박윤진'}
{'배우이름': '고아성'}
{'배우이름': '이솜'}
{'배우이름': '남민우'}
{'배우이름': '하성용'}
{'배우이름': '하윤경'}
{'배우이름': '정재헌'}
{'배우이름': '신햇빛'}
{'배우이름': '이용규'}
{'배우이름': '김성민'}
{'배우이름': '김율'}
{'배우이름': '감소현'}
{'배우이름': '오인실'}
{'배우이름': '김정은'}
{'배우이름': '문승아'}
{'배우이름': '한경화'}
{'배우이름': '박윤진'}


In [104]:
docs = actor.find({'흥행 지수': {'$gte' : 10000 }, '출연영화':'서복' }, {'_id':0, '출연영화':1, '배우이름':1} ).sort('배우이름').limit(3)
for doc in docs:
    print(doc)

{'배우이름': '공유', '출연영화': ['서복', '김종욱 찾기', '82년생 김지영']}
{'배우이름': '공유', '출연영화': ['서복', '김종욱 찾기', '82년생 김지영']}
{'배우이름': '박보검', '출연영화': ['서복']}


### find의 다양한 문법 - or

In [111]:
docs = actor.find({'$or' : [{'출연영화':'도굴'}, {'출연영화':'미나리'}] }, {'_id':0, '출연영화':1, '배우이름':1} ).sort('배우이름')
for doc in docs:
    print(doc)

{'배우이름': '신혜선', '출연영화': ['도굴']}
{'배우이름': '윤여정', '출연영화': ['화녀', '미나리', '지푸라기라도 잡고 싶은 짐승들', '찬실이는 복도 많지', '죽여주는 여자', '그것만이 내 세상']}
{'배우이름': '이제훈', '출연영화': ['도굴', '건축학개론', '파수꾼', '박열']}
{'배우이름': '임원희', '출연영화': ['도굴', '신과 함께-죄와 벌', '신과 함께-인과 연', '천문: 하늘에 묻는다']}
{'배우이름': '조우진', '출연영화': ['도굴', '서복', '국가부도의 날', '강철비']}
{'배우이름': '한예리', '출연영화': ['미나리', '최악의 하루', '더 테이블', '춘몽']}


In [118]:
docs = actor.find({ '흥행지수': {'$gte' : 10000 }, '$or' : [{'출연영화':'도굴'}, {'출연영화':'미나리'}] }, {'_id':0, '출연영화':1, '배우이름':1} ).sort('배우이름')
for doc in docs:
    print(doc)

{'배우이름': '신혜선', '출연영화': ['도굴']}
{'배우이름': '윤여정', '출연영화': ['화녀', '미나리', '지푸라기라도 잡고 싶은 짐승들', '찬실이는 복도 많지', '죽여주는 여자', '그것만이 내 세상']}
{'배우이름': '이제훈', '출연영화': ['도굴', '건축학개론', '파수꾼', '박열']}
{'배우이름': '임원희', '출연영화': ['도굴', '신과 함께-죄와 벌', '신과 함께-인과 연', '천문: 하늘에 묻는다']}
{'배우이름': '조우진', '출연영화': ['도굴', '서복', '국가부도의 날', '강철비']}
{'배우이름': '한예리', '출연영화': ['미나리', '최악의 하루', '더 테이블', '춘몽']}


### find의 다양한 문법 - or
    -nor or

In [121]:
docs = actor.find( { '$nor' : [{'흥행지수': {'$gte':10000 } }, {'흥행지수':{ '$lt': 1000}}]}, {'배우이름':1, '흥행지수':1, '_id':0} ).limit(3)
for doc in docs:
    print(doc)

{'배우이름': '배해선', '흥행지수': 9950}
{'배우이름': '백현진', '흥행지수': 8529}
{'배우이름': '정우', '흥행지수': 8436}


### find의 다양한 문법 - in, nin

In [131]:
docs = actor.find( { '$or':[{'출연영화':{'$nin':['도굴', '사바하']}}, {'흥행지수':{'$gte':1000}}] }, {'배우이름':1, '흥행지수':1, '_id':0} ).limit(3)
for doc in docs:
    print(doc)

{'배우이름': '이제훈', '흥행지수': 100812}
{'배우이름': '조우진', '흥행지수': 88076}
{'배우이름': '신혜선', '흥행지수': 50390}


In [138]:
docs = actor.find( {'$nor' : [{'흥행지수':{'$in':[9182, 8439]}}, {'흥행지수':{'$gt':10000}}] }, {'배우이름':1, '흥행지수':1, '_id':0} ).limit(3)
for doc in docs:
    print(doc)

{'배우이름': '배해선', '흥행지수': 9950}
{'배우이름': '백현진', '흥행지수': 8529}
{'배우이름': '정우', '흥행지수': 8436}


### find의 다양한 문법 -skip, limit
    *skip(n): 검색 결과 n개만큼 건너뜀
    *limit(n): 검색 결과 n개만 표시

In [139]:
docs = actor.find({'흥행지수':{'$gte':10000}}).limit(3)
for doc in docs:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e21c'), '배우이름': '이제훈', '흥행지수': 100812, '출연영화': ['도굴', '건축학개론', '파수꾼', '박열'], '랭킹': '1', '직업': '배우', '생년월일': '1984-07-04', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.leejehoon', '신장/체중': '176cm', '학교': '한국예술종합학교 연극원 연기과', '특기': '댄스, 노래, 무술', '소속사': '㈜사람엔터테인먼트'}
{'_id': ObjectId('608ffdadbc196d809d52e21d'), '배우이름': '조우진', '흥행지수': 88076, '출연영화': ['도굴', '서복', '국가부도의 날', '강철비'], '랭킹': '2', '다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'}
{'_id': ObjectId('608ffdadbc196d809d52e21e'), '배우이름': '신혜선', '흥행지수': 50390, '출연영화': ['도굴'], '랭킹': '3', '직업': '배우', '생년월일': '1989-00-00', '성별': '여', '홈페이지': 'https://www.facebook.com/shs831 https://www.instagram.com/shinhs831/'}


In [141]:
docs = actor.find({'흥행지수':{'$gte':10000}}).skip(3).limit(3)
for doc in docs:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e21f'), '배우이름': '한예리', '흥행지수': 47424, '출연영화': ['미나리', '최악의 하루', '더 테이블', '춘몽'], '랭킹': '4', '직업': '배우', '생년월일': '1984-12-23', '성별': '여', '홈페이지': 'https://www.facebook.com/saram.hanyeri', '학교': '한국예술종합학교'}
{'_id': ObjectId('608ffdadbc196d809d52e220'), '배우이름': '김영광', '흥행지수': 44812, '출연영화': ['미션 파서블'], '랭킹': '5', '직업': '배우', '생년월일': '1987-01-11', '성별': '남', '홈페이지': 'https://twitter.com/aksakfn12https://www.instagram.com/aksakfn12/'}
{'_id': ObjectId('608ffdadbc196d809d52e221'), '배우이름': '윤여정', '흥행지수': 39673, '출연영화': ['화녀', '미나리', '지푸라기라도 잡고 싶은 짐승들', '찬실이는 복도 많지', '죽여주는 여자', '그것만이 내 세상'], '랭킹': '6', '원어명': '윤여정', '직업': '배우', '생년월일': '1947-06-19', '성별': '여', '학교': '한양대학교 국문학 (중퇴)'}


### find의 다양한 문법 -list 검색

In [143]:
docs = actor.find({'$or': [{"출연영화":"미나리"}, {"출연영화":"도굴"}]})
for doc in docs:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e21c'), '배우이름': '이제훈', '흥행지수': 100812, '출연영화': ['도굴', '건축학개론', '파수꾼', '박열'], '랭킹': '1', '직업': '배우', '생년월일': '1984-07-04', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.leejehoon', '신장/체중': '176cm', '학교': '한국예술종합학교 연극원 연기과', '특기': '댄스, 노래, 무술', '소속사': '㈜사람엔터테인먼트'}
{'_id': ObjectId('608ffdadbc196d809d52e21d'), '배우이름': '조우진', '흥행지수': 88076, '출연영화': ['도굴', '서복', '국가부도의 날', '강철비'], '랭킹': '2', '다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'}
{'_id': ObjectId('608ffdadbc196d809d52e21e'), '배우이름': '신혜선', '흥행지수': 50390, '출연영화': ['도굴'], '랭킹': '3', '직업': '배우', '생년월일': '1989-00-00', '성별': '여', '홈페이지': 'https://www.facebook.com/shs831 https://www.instagram.com/shinhs831/'}
{'_id': ObjectId('608ffdadbc196d809d52e21f'), '배우이름': '한예리', '흥행지수': 47424, '출연영화': ['미나리', '최악의 하루', '더 테이블', '춘몽'], '랭킹': '4', '직업': '배우', '생년월일': '1984-12-23', '성별': '여', '홈페이지': 'https://www.facebook.com/saram.hanyeri', '학교': '한국예술종합학교'}
{'_id': ObjectId('608ffdadbc196d809

In [146]:
docs = actor.find({'$and': [{"출연영화":"미나리"}, {"출연영화":"화녀"}]})
for doc in docs:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e221'), '배우이름': '윤여정', '흥행지수': 39673, '출연영화': ['화녀', '미나리', '지푸라기라도 잡고 싶은 짐승들', '찬실이는 복도 많지', '죽여주는 여자', '그것만이 내 세상'], '랭킹': '6', '원어명': '윤여정', '직업': '배우', '생년월일': '1947-06-19', '성별': '여', '학교': '한양대학교 국문학 (중퇴)'}


In [147]:
# docs = actor.find( {'출연영화': ['극한직업', '사바하']}) # 딱 극한직업 사바하만 들어간 배우를 고름
docs = actor.find( {'출연영화': { '$all': ['도굴', '건축학개론']}})
for doc in docs:
    print(doc)


{'_id': ObjectId('608ffdadbc196d809d52e21c'), '배우이름': '이제훈', '흥행지수': 100812, '출연영화': ['도굴', '건축학개론', '파수꾼', '박열'], '랭킹': '1', '직업': '배우', '생년월일': '1984-07-04', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.leejehoon', '신장/체중': '176cm', '학교': '한국예술종합학교 연극원 연기과', '특기': '댄스, 노래, 무술', '소속사': '㈜사람엔터테인먼트'}


In [148]:
docs = actor.find({'출연영화.0': '도굴'})
for doc in docs:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e21c'), '배우이름': '이제훈', '흥행지수': 100812, '출연영화': ['도굴', '건축학개론', '파수꾼', '박열'], '랭킹': '1', '직업': '배우', '생년월일': '1984-07-04', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.leejehoon', '신장/체중': '176cm', '학교': '한국예술종합학교 연극원 연기과', '특기': '댄스, 노래, 무술', '소속사': '㈜사람엔터테인먼트'}
{'_id': ObjectId('608ffdadbc196d809d52e21d'), '배우이름': '조우진', '흥행지수': 88076, '출연영화': ['도굴', '서복', '국가부도의 날', '강철비'], '랭킹': '2', '다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'}
{'_id': ObjectId('608ffdadbc196d809d52e21e'), '배우이름': '신혜선', '흥행지수': 50390, '출연영화': ['도굴'], '랭킹': '3', '직업': '배우', '생년월일': '1989-00-00', '성별': '여', '홈페이지': 'https://www.facebook.com/shs831 https://www.instagram.com/shinhs831/'}
{'_id': ObjectId('608ffdadbc196d809d52e222'), '배우이름': '임원희', '흥행지수': 25245, '출연영화': ['도굴', '신과 함께-죄와 벌', '신과 함께-인과 연', '천문: 하늘에 묻는다'], '랭킹': '7', '직업': '배우', '생년월일': '1970-10-11', '성별': '남', '신장/체중': '175cm, 68kg', '학교': '서울예술대학 연극', '소속사': '빌리지엔터테인먼트'}


In [150]:
docs = actor.find({'출연영화':{ '$size':5}})
for doc in docs:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e22f'), '배우이름': '김종수', '흥행지수': 12797, '출연영화': ['삼진그룹 영어토익반', '오케이 마담', '시동', '협상', '야구소녀'], '랭킹': '20', '직업': '배우', '생년월일': '1964-11-30', '성별': '남', '홈페이지': 'http://www.artistcompany.co.kr/portfolio-item/kim-jong-soo/', '소속사': '아티스트컴퍼니'}
{'_id': ObjectId('608ffdadbc196d809d52e236'), '배우이름': '김희원', '흥행지수': 10456, '출연영화': ['이웃사촌', '담보', '불한당: 나쁜 놈들의 세상', '국제수사', '판소리 복서'], '랭킹': '27', '직업': '배우', '생년월일': '1971-01-10', '성별': '남', '신장/체중': '180cm', '학교': '서울예술대학 연극과', '소속사': 'DMS커뮤니케이션즈'}
{'_id': ObjectId('608ffdadbc196d809d52e249'), '배우이름': '유연석', '흥행지수': 4371, '출연영화': ['강철비2: 정상회담', '건축학개론', '늑대소년', '새해전야', '제보자'], '랭킹': '46', '다른 이름': '안연석', '직업': '배우', '생년월일': '1984-04-11', '성별': '남', '홈페이지': 'https://twitter.com/yeonseok411https://www.instagram.com/yoo_yeonseok/', '소속사': '이야기엔터테인먼트'}
{'_id': ObjectId('608ffdadbc196d809d52e25c'), '배우이름': '류현경', '흥행지수': 2061, '출연영화': ['아이', '기도하는 남자', '태양은 없다', '제보자', '호랑이보다 무서운 겨울손님'], '랭킹': '65', '직업': '배우', '생년월일

### find의 다양한 문법 ( elemMatch)
* 적어도 한개 이상의 리스트 요소가 복수 개의 조건을 동시에 만족하는 경우

In [152]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn['cine21']
elemmatch_sample = actor_db['sample']

In [154]:
elemmatch_sample.insert_many([
    {'results': [82, 85, 88]},
    {'results': [75, 88, 91]}
]
)

In [163]:
docs = elemmatch_sample.find({'results': {'$gte':90, '$lt':85}})
for doc in docs:
    print(doc)

{'_id': ObjectId('60900630bc196d809d52e2ab'), 'results': [75, 88, 91]}


In [164]:
docs = elemmatch_sample.find({'results': {'$elemMatch': {'$gte':75, '$lt':80}}})
for doc in docs:
    print(doc)

{'_id': ObjectId('60900630bc196d809d52e2ab'), 'results': [75, 88, 91]}


In [165]:
docs = actor.find({'직업':'가수'}).sort('흥행지수', pymongo.DESCENDING).limit(10)
for doc in docs:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e255'), '배우이름': '찬열', '흥행지수': 3059, '출연영화': ['더 박스'], '랭킹': '58', '다른 이름': '박찬열; 엑소;EXO', '직업': '가수', '생년월일': '1992-11-27', '성별': '남', '홈페이지': 'https://instagram.com/real__pcy/', '소속사': 'SM엔터테인먼트'}
{'_id': ObjectId('608ffdadbc196d809d52e257'), '배우이름': '송가인', '흥행지수': 2670, '출연영화': ['송가인 더 드라마'], '랭킹': '60', '다른 이름': '조은심', '직업': '가수', '생년월일': '1986-12-26', '성별': '여', '홈페이지': 'http://instagram.com/songgain_http://www.facebook.com/eunsim.jo.9'}
{'_id': ObjectId('608ffdadbc196d809d52e25e'), '배우이름': '조현', '흥행지수': 1896, '출연영화': ['최면'], '랭킹': '67', '원어명': '신지원', '다른 이름': '베리굿', '직업': '가수', '생년월일': '1996-04-14', '성별': '여', '홈페이지': 'http://www.vlive.tv/channels/E982B5'}
{'_id': ObjectId('608ffdadbc196d809d52e25f'), '배우이름': '수영', '흥행지수': 1864, '출연영화': ['새해전야'], '랭킹': '68', '다른 이름': '최수영; 소녀시대; girlsgeneration; girls generation', '직업': '가수', '생년월일': '1990-02-10', '성별': '여', '홈페이지': 'https://www.instagram.com/hotsootuff/', '신장/체중': '170cm', '특기': '일본어', '소속사':

In [169]:
docs = actor.find({'출연영화': {'$in':['국가부도의 날']}}).sort('흥행지수', pymongo.DESCENDING).limit(10)
for doc in docs:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e21d'), '배우이름': '조우진', '흥행지수': 88076, '출연영화': ['도굴', '서복', '국가부도의 날', '강철비'], '랭킹': '2', '다른 이름': '조신제', '직업': '배우', '생년월일': '1979-01-16', '성별': '남'}
{'_id': ObjectId('608ffdadbc196d809d52e22b'), '배우이름': '염혜란', '흥행지수': 16046, '출연영화': ['내일의 기억', '이웃사촌', '아이', '빛과 철', '비밀의 정원', '국가부도의 날'], '랭킹': '16', '직업': '배우', '생년월일': '1976-00-00', '성별': '여'}
{'_id': ObjectId('608ffdadbc196d809d52e235'), '배우이름': '김혜수', '흥행지수': 10952, '출연영화': ['내가 죽던 날', '국가부도의 날', '도둑들'], '랭킹': '26', '직업': '배우', '생년월일': '1970-09-05', '성별': '여', '신장/체중': '171cm, 49kg', '학교': '동국대 연극영화 - 성균관대언론정보대학원 석사', '취미': '영화 감상, 사진집 모으기, 태권도, 수영, 테니스, 볼링', '특기': '태권도'}
{'_id': ObjectId('608ffdadbc196d809d52e263'), '배우이름': '유아인', '흥행지수': 1725, '출연영화': ['소리도 없이', '국가부도의 날'], '랭킹': '72', '다른 이름': '엄홍식', '직업': '배우', '생년월일': '1986-10-06', '성별': '남', '홈페이지': 'https://www.instagram.com/hongsick/https://twitter.com/seeksik', '신장/체중': '178cm, 60kg', '학교': '단국대학교 연극', '취미': '피아노 연주, 인터넷 게임, 영화, 음악감상', '

## 2. mongodb 인덱스(Index)
- 검색을 더 빠르게 수행하고자 만든 추가적인 data structure
    - index 데이터 구조가 없다면, 컬렉션에 있는 데이터를 하나씩 조회하는 방식으로 검색이 됨

### 2.1 기본 인덱스 _id
- 모든 mongodb 컬렉션은 기본적으로 _id 필드를 기반으로 기본 인덱스가 생성됨

### 2.2 single(단일) 필드 인덱스

#### index 생성:creatae_index()

In [174]:
actor.create_index('배우이름')

'배우이름_1'

In [197]:
docs = actor.find({'배우이름':'이제훈'})
for doc in docs:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e21c'), '배우이름': '이제훈', '흥행지수': 100812, '출연영화': ['도굴', '건축학개론', '파수꾼', '박열'], '랭킹': '1', '직업': '배우', '생년월일': '1984-07-04', '성별': '남', '홈페이지': 'https://www.facebook.com/saram.leejehoon', '신장/체중': '176cm', '학교': '한국예술종합학교 연극원 연기과', '특기': '댄스, 노래, 무술', '소속사': '㈜사람엔터테인먼트'}


In [173]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}}

- key: ('필드명', direction)
    - direction
        - pymongo.ASCENDING = 1
        - pymongo.DECENDING = -1
        - pymongo.TEXT = 'text'

### 전체 인덱스 삭제

In [235]:
actor.drop_indexes()

### 특정 인덱스 삭제

In [176]:
actor.drop_index([('배우이름', 1)])

In [224]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 '직업_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('직업', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

#### 여러개의 single index 생성가능

In [179]:
actor.create_index('배우이름')

'배우이름_1'

In [180]:
actor.create_index('랭킹')

'랭킹_1'

In [181]:
actor.create_index('흥행지수')

'흥행지수_1'

In [187]:
actor.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 '배우이름_1': {'v': 2, 'key': [('배우이름', 1)]},
 '랭킹_1': {'v': 2, 'key': [('랭킹', 1)]},
 '흥행지수_1': {'v': 2, 'key': [('흥행지수', 1)]},
 '직업_-1': {'v': 2, 'key': [('직업', -1)]},
 '직업_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('직업', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [184]:
actor.create_index([('직업', pymongo.DESCENDING)])

'직업_-1'

#### 부분 문자열 검색을 위해, text index 생성

In [228]:
actor.create_index([('출연영화', 'text')]) # 컬랙선당 하나만 존재가능

'출연영화_text'

#### 부분 문자열 검색: $text

In [231]:
docs = actor.find({'$text': {'$search': '신과'}})
for doc in docs: 
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e283'), '배우이름': '장광', '흥행지수': 810, '출연영화': ['가나의 혼인잔치: 언약', '아수라도', '신과 함께-죄와 벌', '신과 함께-인과 연', '강철비2: 정상회담', '신세계'], '랭킹': '104', '직업': '배우', '생년월일': '1952-01-05', '성별': '남'}
{'_id': ObjectId('608ffdadbc196d809d52e253'), '배우이름': '김향기', '흥행지수': 3137, '출연영화': ['아이', '신과 함께-죄와 벌', '신과 함께-인과 연', '늑대소년'], '랭킹': '56', '직업': '배우', '생년월일': '2000-08-09', '성별': '여', '홈페이지': 'https://twitter.com/smell2001', '특기': '애교 부리기, 피아노 치기'}
{'_id': ObjectId('608ffdadbc196d809d52e222'), '배우이름': '임원희', '흥행지수': 25245, '출연영화': ['도굴', '신과 함께-죄와 벌', '신과 함께-인과 연', '천문: 하늘에 묻는다'], '랭킹': '7', '직업': '배우', '생년월일': '1970-10-11', '성별': '남', '신장/체중': '175cm, 68kg', '학교': '서울예술대학 연극', '소속사': '빌리지엔터테인먼트'}
{'_id': ObjectId('608ffdadbc196d809d52e28a'), '배우이름': '조한철', '흥행지수': 682, '출연영화': ['세자매', '신과 함께-인과 연', '정직한 후보', '배심원들', '백두산', '국가부도의 날'], '랭킹': '111', '직업': '배우', '생년월일': '1973-05-13', '성별': '남'}
{'_id': ObjectId('608ffdadbc196d809d52e23d'), '배우이름': '오달수', '흥행지수': 7355, '출연영화': [

### 2.3 Compount(복합) 필드 인덱스
    - 최대 31개의 필드로 만들수 있음

In [237]:
actor.create_index([('출연영화', pymongo.TEXT), ('직업', pymongo.TEXT), ('흥행지수', pymongo.DESCENDING)])

'출연영화_text_직업_text_흥행지수_-1'

In [241]:
docs = actor.find({'$text': {'$search': '신과'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e283'), '배우이름': '장광', '흥행지수': 810, '출연영화': ['가나의 혼인잔치: 언약', '아수라도', '신과 함께-죄와 벌', '신과 함께-인과 연', '강철비2: 정상회담', '신세계'], '랭킹': '104', '직업': '배우', '생년월일': '1952-01-05', '성별': '남'}
{'_id': ObjectId('608ffdadbc196d809d52e253'), '배우이름': '김향기', '흥행지수': 3137, '출연영화': ['아이', '신과 함께-죄와 벌', '신과 함께-인과 연', '늑대소년'], '랭킹': '56', '직업': '배우', '생년월일': '2000-08-09', '성별': '여', '홈페이지': 'https://twitter.com/smell2001', '특기': '애교 부리기, 피아노 치기'}
{'_id': ObjectId('608ffdadbc196d809d52e222'), '배우이름': '임원희', '흥행지수': 25245, '출연영화': ['도굴', '신과 함께-죄와 벌', '신과 함께-인과 연', '천문: 하늘에 묻는다'], '랭킹': '7', '직업': '배우', '생년월일': '1970-10-11', '성별': '남', '신장/체중': '175cm, 68kg', '학교': '서울예술대학 연극', '소속사': '빌리지엔터테인먼트'}
{'_id': ObjectId('608ffdadbc196d809d52e28a'), '배우이름': '조한철', '흥행지수': 682, '출연영화': ['세자매', '신과 함께-인과 연', '정직한 후보', '배심원들', '백두산', '국가부도의 날'], '랭킹': '111', '직업': '배우', '생년월일': '1973-05-13', '성별': '남'}
{'_id': ObjectId('608ffdadbc196d809d52e23d'), '배우이름': '오달수', '흥행지수': 7355, '출연영화': [

### 2.4 다양한 문자열 검색

In [242]:
result = actor.find({'출연영화': {'$regex' : '함께'}})
for doc in result:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e222'), '배우이름': '임원희', '흥행지수': 25245, '출연영화': ['도굴', '신과 함께-죄와 벌', '신과 함께-인과 연', '천문: 하늘에 묻는다'], '랭킹': '7', '직업': '배우', '생년월일': '1970-10-11', '성별': '남', '신장/체중': '175cm, 68kg', '학교': '서울예술대학 연극', '소속사': '빌리지엔터테인먼트'}
{'_id': ObjectId('608ffdadbc196d809d52e23d'), '배우이름': '오달수', '흥행지수': 7355, '출연영화': ['이웃사촌', '신과 함께-죄와 벌', '올드보이', '박쥐', '친절한 금자씨', '싸이보그지만 괜찮아'], '랭킹': '34', '직업': '배우', '생년월일': '1968-06-15', '성별': '남', '신장/체중': '176cm, 68kg', '학교': '동의대학교 공업디자인 (중퇴)', '소속사': '빌리지엔터테인먼트'}
{'_id': ObjectId('608ffdadbc196d809d52e253'), '배우이름': '김향기', '흥행지수': 3137, '출연영화': ['아이', '신과 함께-죄와 벌', '신과 함께-인과 연', '늑대소년'], '랭킹': '56', '직업': '배우', '생년월일': '2000-08-09', '성별': '여', '홈페이지': 'https://twitter.com/smell2001', '특기': '애교 부리기, 피아노 치기'}
{'_id': ObjectId('608ffdadbc196d809d52e283'), '배우이름': '장광', '흥행지수': 810, '출연영화': ['가나의 혼인잔치: 언약', '아수라도', '신과 함께-죄와 벌', '신과 함께-인과 연', '강철비2: 정상회담', '신세계'], '랭킹': '104', '직업': '배우', '생년월일': '1952-01-05', '성별': '남'}
{'_id': Ob

In [243]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn['cine21']
text_collection = actor_db['text_collection']

In [244]:
text_collection.insert_many(
    [
        {'name': 'Java Hut', 'description': 'Coffee and cakes', 'ranking':1},
        {'name': 'Burger Buns', 'description': 'Java hamburgers', 'ranking':2},
        {'name': 'Coffee shop', 'description': 'Just coffee', 'ranking':3},
        {'name': 'Clothes Clothes Clothes', 'description': 'Discount clothing', 'ranking':4},
        {'name': 'Java shopping', 'description': 'Indonesian goods', 'ranking':5}
    ]
)

In [247]:
docs = text_collection.find({'name':{'$regex': 'sh.*'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('6090d7dcbc196d809d52e2af'), 'name': 'Coffee shop', 'description': 'Just coffee', 'ranking': 3}
{'_id': ObjectId('6090d7dcbc196d809d52e2b1'), 'name': 'Java shopping', 'description': 'Indonesian goods', 'ranking': 5}


In [248]:
text_collection.create_index([('name', pymongo.TEXT)])

'name_text'

In [249]:
text_collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'name_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('name', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [250]:
docs = text_collection.find({'$text':{'$search':'coffee'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('6090d7dcbc196d809d52e2af'), 'name': 'Coffee shop', 'description': 'Just coffee', 'ranking': 3}


In [251]:
docs = text_collection.find({'$text':{'$search':'java coffee shop'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('6090d7dcbc196d809d52e2af'), 'name': 'Coffee shop', 'description': 'Just coffee', 'ranking': 3}
{'_id': ObjectId('6090d7dcbc196d809d52e2b1'), 'name': 'Java shopping', 'description': 'Indonesian goods', 'ranking': 5}
{'_id': ObjectId('6090d7dcbc196d809d52e2ad'), 'name': 'Java Hut', 'description': 'Coffee and cakes', 'ranking': 1}


In [252]:
docs = text_collection.find({'$text':{'$search':'\"coffee shop\"'}}) # 대소문자 구별 안함
for doc in docs:
    print(doc)

{'_id': ObjectId('6090d7dcbc196d809d52e2af'), 'name': 'Coffee shop', 'description': 'Just coffee', 'ranking': 3}


In [253]:
docs = text_collection.find({'$text':{'$search':'Coffee', '$caseSensitive': True}})
for doc in docs:
    print(doc)

{'_id': ObjectId('6090d7dcbc196d809d52e2af'), 'name': 'Coffee shop', 'description': 'Just coffee', 'ranking': 3}


In [254]:
actor.drop_indexes()

In [255]:
actor.create_index([('학교', 'text')])

'학교_text'

In [261]:
docs = actor.find({'$text':{'$search':'중앙대학교'}})
for doc in docs:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e267'), '배우이름': '김도훈', '흥행지수': 1580, '출연영화': ['최면'], '랭킹': '76', '직업': '배우', '생년월일': '1998-09-15', '성별': '남', '신장/체중': '182cm, 70kg', '학교': '중앙대학교'}
{'_id': ObjectId('608ffdadbc196d809d52e256'), '배우이름': '정희태', '흥행지수': 2812, '출연영화': ['이웃사촌'], '랭킹': '59', '직업': '배우', '생년월일': '1974-07-05', '성별': '남', '홈페이지': 'https://twitter.com/hite9999https://www.facebook.com/profile.php?id=100001688391574', '신장/체중': '180cm, 72kg', '학교': '중앙대학교'}
{'_id': ObjectId('608ffdadbc196d809d52e254'), '배우이름': '이연희', '흥행지수': 3106, '출연영화': ['새해전야'], '랭킹': '57', '직업': '배우', '생년월일': '1988-01-09', '성별': '여', '홈페이지': 'https://www.facebook.com/leeyeonhee', '신장/체중': '170cm, 46kg', '학교': '중앙대학교 연극영화학', '취미': '영화감상, 독서', '특기': '수영', '소속사': 'SM엔터테인먼트'}
{'_id': ObjectId('608ffdadbc196d809d52e241'), '배우이름': '김병철', '흥행지수': 6749, '출연영화': ['이웃사촌'], '랭킹': '38', '직업': '배우', '생년월일': '1974-00-00', '성별': '남', '학교': '중앙대학교 연극학과'}
{'_id': ObjectId('608ffdadbc196d809d52e230'), '배우이름': '강하늘', '흥행지수':

In [263]:
docs = actor.find({'학교':{'$regex':'중앙대학교'}}).sort('흥행지수',pymongo.DESCENDING).limit(10)
for doc in docs:
    print(doc)

{'_id': ObjectId('608ffdadbc196d809d52e22d'), '배우이름': '김강우', '흥행지수': 14757, '출연영화': ['내일의 기억', '새해전야'], '랭킹': '18', '직업': '배우', '생년월일': '1978-07-11', '성별': '남', '홈페이지': 'https://www.facebook.com/cjeskangwoo', '신장/체중': '180cm, 70kg', '학교': '중앙대학교 연극영화학', '취미': '영화감상, 운동', '특기': '검도', '소속사': '나무액터스'}
{'_id': ObjectId('608ffdadbc196d809d52e230'), '배우이름': '강하늘', '흥행지수': 11867, '출연영화': ['비와 당신의 이야기', '동주'], '랭킹': '21', '다른 이름': '김하늘', '직업': '배우', '생년월일': '1990-02-21', '성별': '남', '홈페이지': 'http://weibo.com/galpos3?is_hot=1', '신장/체중': '181cm, 70kg', '학교': '중앙대학교 연극학과'}
{'_id': ObjectId('608ffdadbc196d809d52e241'), '배우이름': '김병철', '흥행지수': 6749, '출연영화': ['이웃사촌'], '랭킹': '38', '직업': '배우', '생년월일': '1974-00-00', '성별': '남', '학교': '중앙대학교 연극학과'}
{'_id': ObjectId('608ffdadbc196d809d52e254'), '배우이름': '이연희', '흥행지수': 3106, '출연영화': ['새해전야'], '랭킹': '57', '직업': '배우', '생년월일': '1988-01-09', '성별': '여', '홈페이지': 'https://www.facebook.com/leeyeonhee', '신장/체중': '170cm, 46kg', '학교': '중앙대학교 연극영화학', '취미': '영화감상, 독서', '특